In [1]:
import requests
import base64
from pathlib import Path
import os

def encode_audio_to_base64(path: Path):
    with open(path, "rb") as audio_file:
        return base64.b64encode(audio_file.read()).decode()

def decode_base64_to_audio(b64_str: str, output_path: Path):
    audio_bytes = base64.b64decode(b64_str)
    with open(output_path, "wb") as audio_file:
        audio_file.write(audio_bytes)

# Define multiple pairs of input audio files
audio_pairs = [
    ("data/source1.wav", "data/target1.wav"),
    ("data/source2.wav", "data/target2.wav"),
    ("data/source3.wav", "data/target3.wav"),
    # Add as many pairs as needed
]

# Prepare payload with multiple samples
payload = {
    "samples": [
        {
            "source_base64": encode_audio_to_base64(Path(src)),
            "target_base64": encode_audio_to_base64(Path(tgt))
        }
        for src, tgt in audio_pairs
    ]
}

# Make the request to the FastAPI endpoint
response = requests.post("http://127.0.0.1:8000/convert-voices", json=payload)

os.makedirs("./output",exist_ok=True)

if response.status_code == 200:
    outputs = response.json()
    for idx, output in enumerate(outputs):
        output_audio_path = Path(f"./output/output_converted_{idx}.wav")
        decode_base64_to_audio(output["output_base64"], output_audio_path)
        print(f"Audio saved to: {output_audio_path}")
else:
    print("Error:", response.status_code, response.text)

# if response.status_code == 200:
#     outputs = response.json()
#     for idx, output in enumerate(outputs):
#         print(f"Audio saved to {idx}: {output['output_base64']}")
# else:
#     print("Error:", response.status_code, response.text)


Audio saved to: output/output_converted_0.wav
Audio saved to: output/output_converted_1.wav
Audio saved to: output/output_converted_2.wav


In [6]:
from IPython.display import Audio

Audio("./output/output_converted_1.wav")